**How many COVID cases will there be?**

Factors:
* COVID case growth rates
* Vaccines timeline (getting EUA, starting distribution, achieving sufficient scale)
* Delay between achievement and the media article

In [1]:
import math
import random

import pandas as pd
import numpy as np

from scipy import stats
from datetime import datetime, timedelta


# Model variables
GROWTH_DERIVATIVE_FACTOR_LOW = 0.5
GROWTH_DERIVATIVE_FACTOR_HIGH = 3.0

MONTHLY_GROWTH_MODIFIER_LOW = 0.5
MONTHLY_GROWTH_MODIFIER_HIGH = 3.0

DAYS_UNTIL_PEAK_GROWTH_LOW = -30
DAYS_UNTIL_PEAK_GROWTH_HIGH = 60

MISSING_INFECTION_RATE_LOW = 5
MISSING_INFECTION_RATE_HIGH = 30

RANDOM_CASE_FLUCTUATION_LOW = -8000
RANDOM_CASE_FLUCTUATION_HIGH = 40000

DAILY_FOURTH_WAVE_CHANCE = 0.6 / 365
FOURTH_WAVE_LENGTH_DAYS_LOW = 30
FOURTH_WAVE_LENGTH_DAYS_HIGH = 120

VACCINE_DELAY_DAYS_LOW = 7
VACCINE_DELAY_DAYS_HIGH = 28

NEWS_ARTICLE_DELAY_DAYS_LOW = -5 # Earlier than it ought to be based on mistaken reporting, projections, estimations
NEWS_ARTICLE_DELAY_DAYS_HIGH = 15

million = 1000000

# These are set to roughly curve fit the estimates from
# https://www.metaculus.com/questions/4066/10m-are-administered-an-efficacious-vaccine/
# https://www.metaculus.com/questions/5727/100m-people-vaccinated-against-covid-19/
# https://www.metaculus.com/questions/5767/1bn-people-vaccinated-against-covid-19/
MONTHLY_VACCINATION_DAILY_AMOUNTS = [np.round(20 * million * (1/30) * (1/12), 0),
                                     np.round(20 * million * (1/30) * (1/12), 0),
                                     np.round(100 * million * (1/30) * (1/12), 0),
                                     np.round(150 * million * (1/30) * (1/12), 0),
                                     np.round(200 * million * (1/30) * (1/12), 0),
                                     np.round(250 * million * (1/30) * (1/12), 0),
                                     np.round(300 * million * (1/30) * (1/12), 0),
                                     np.round(400 * million * (1/30) * (1/12), 0),
                                     np.round(700 * million * (1/30) * (1/12), 0),
                                     np.round(800 * million * (1/30) * (1/12), 0),
                                     np.round(900 * million * (1/30) * (1/12), 0),
                                     np.round(1000 * million * (1/30) * (1/12), 0),
                                     np.round(1100 * million * (1/30) * (1/12), 0)]

VACCINE_MODIFIER_LOW = 0.5
VACCINE_MODIFIER_HIGH = 4.0

# Simulation parameters
N_SCENARIOS = 1000
SIM_START_DATE = '2020-11-28'
SIM_END_DATE = '2022-01-01'
CURRENT_COVID_CASES = 61.8 * million
CURRENT_COVID_GROWTH = 571428
CURRENT_COVID_DERIVATIVE = 4000
Q4_2020_START_COVID = 26.02 * million

In [2]:
def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<1.0%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def get_metaculus_range(rangex, minx=None, maxx=None):
    lower25, mean, upper75 = np.percentile(rangex, [25, 50, 75])
    results = {}
    results['lower25'] = lower25
    results['mean'] = mean
    results['upper75'] = upper75
    if minx:
        results['minx'] = print_pct(sum([x < minx for x in rangex]) / len(rangex))
    if maxx:
        results['maxx'] = print_pct(sum([x > maxx for x in rangex]) / len(rangex))
    return results
    

def normal_sample(low, high, interval):
    if (low > high) or (high < low):
        raise ValueError
    if low == high:
        return low
    else:
        mu = (high + low) / 2
        cdf_value = 0.5 + 0.5 * interval
        normed_sigma = stats.norm.ppf(cdf_value)
        sigma = (high - mu) / normed_sigma
        return np.random.normal(mu, sigma)

    
def lognormal_sample(low, high, interval):
    if (low > high) or (high < low):
        raise ValueError
    if low == high:
        return low
    else:
        log_low = np.log(low)
        log_high = np.log(high)
        mu = (log_high + log_low) / 2
        cdf_value = 0.5 + 0.5 * interval
        normed_sigma = stats.norm.ppf(cdf_value)
        sigma = (log_high - mu) / normed_sigma
        return np.random.lognormal(mu, sigma)

In [3]:
days_until_sim_end_date = (datetime.strptime(SIM_END_DATE, '%Y-%m-%d') - datetime.strptime(SIM_START_DATE, '%Y-%m-%d')).days + 1

dec_7_2020_covid_collector = []
q4_2020_covid_collector = []
y2020_covid_collector = []
q1_2021_covid_collector = []
jan_covid_collector = []
feb_covid_collector = []
mar_covid_collector = []
apr_covid_collector = []
may_covid_collector = []
jun_covid_collector = []
jul_covid_collector = []
aug_covid_collector = []
sep_covid_collector = []
oct_covid_collector = []
nov_covid_collector = []
dec_covid_collector = []
dec_2020_vaccinated_collector = []
jan_vaccinated_collector = []
feb_vaccinated_collector = []
mar_vaccinated_collector = []
apr_vaccinated_collector = []
may_vaccinated_collector = []
jun_vaccinated_collector = []
jul_vaccinated_collector = []
aug_vaccinated_collector = []
sep_vaccinated_collector = []
oct_vaccinated_collector = []
nov_vaccinated_collector = []
dec_vaccinated_collector = []


for n in range(N_SCENARIOS):
    fourth_wave = False
    today = datetime.today()
    
    covid = CURRENT_COVID_CASES
    covid_growth = CURRENT_COVID_GROWTH
    
    growth_derivative_factor = np.round(normal_sample(GROWTH_DERIVATIVE_FACTOR_LOW, GROWTH_DERIVATIVE_FACTOR_HIGH, 0.8) + 1, 1)
    if growth_derivative_factor < 0.3:
        growth_derivative_factor = 0.3
        
    monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
    if monthly_growth_modifier < 0.5:
        monthly_growth_modifier = 0.5
     
    peak_growth_days = np.round(normal_sample(DAYS_UNTIL_PEAK_GROWTH_LOW, DAYS_UNTIL_PEAK_GROWTH_HIGH, 0.8), 0)
    if peak_growth_days < 0:
        peak_growth_days = 0
    peak_growth_date = (today + timedelta(days=peak_growth_days)).date()
    
    fourth_wave_days_length = np.round(lognormal_sample(FOURTH_WAVE_LENGTH_DAYS_LOW, FOURTH_WAVE_LENGTH_DAYS_HIGH, 0.8), 0)
    if fourth_wave_days_length < 1:
        fourth_wave_days_length = 1
        
    missing_infection_rate = np.round(lognormal_sample(MISSING_INFECTION_RATE_LOW, MISSING_INFECTION_RATE_HIGH, 0.8), 1)
    if missing_infection_rate < 1:
        missing_infection_rate = 1
        
    vaccine_modifier = np.round(lognormal_sample(VACCINE_MODIFIER_LOW, VACCINE_MODIFIER_HIGH, 0.8), 1)
    if vaccine_modifier < 0.2:
        vaccine_modifier = 0.2
    
    vaccine_delay_days = np.round(normal_sample(VACCINE_DELAY_DAYS_LOW, VACCINE_DELAY_DAYS_HIGH, 0.8), 1)
    
    news_article_delay_days = np.round(normal_sample(NEWS_ARTICLE_DELAY_DAYS_LOW, NEWS_ARTICLE_DELAY_DAYS_HIGH, 0.8), 1)
    
    vaccine_rate = 0
    total_vaccinated = 0
    
    verbose = (n <= 4) or (n % 10 == 0)
    really_verbose = (n <= 4)
    
    if verbose:
        print('-')
        print('-')
        print('## SCENARIO {}/{} ##'.format(n + 1, N_SCENARIOS))
        print('Growth derivative factor: {}'.format(growth_derivative_factor))
        print('Vaccine modifier: {}'.format(vaccine_modifier))
        print('Peak growth on: {}'.format(peak_growth_date))
        print('Missing infection rate: {}'.format(missing_infection_rate))
        print('Vaccine delay days: {}'.format(vaccine_delay_days))
        print('News article delay days: {}'.format(news_article_delay_days))

    for day in range(days_until_sim_end_date):
        date = str((today + timedelta(days = day)).date())
        if really_verbose:
            print('Day {}: {}'.format(day, date))
            
        # Grow vaccines
        days_since_vaccine_start = (today + timedelta(days = day) - datetime(2020, 12, 15) - timedelta(days = vaccine_delay_days) - timedelta(days = news_article_delay_days)).days
        if days_since_vaccine_start >= 0:
            i = math.floor(days_since_vaccine_start / 30)
            if i >= len(MONTHLY_VACCINATION_DAILY_AMOUNTS):
                i = len(MONTHLY_VACCINATION_DAILY_AMOUNTS) - 1
            daily_vaccines = MONTHLY_VACCINATION_DAILY_AMOUNTS[i]
            old_vaccine_rate = vaccine_rate
            vaccine_rate = np.round(np.random.normal(daily_vaccines * vaccine_modifier,
                                                     daily_vaccines * vaccine_modifier), 0)
            if vaccine_rate < old_vaccine_rate:
                vaccine_rate = old_vaccine_rate
            total_vaccinated = np.round(total_vaccinated + vaccine_rate, 0)
            
        if really_verbose:
            print('...Total vaccinated now {} (+{})'.format(total_vaccinated, vaccine_rate))
            
            
        # Grow COVID
        if day > peak_growth_days and (not fourth_wave or day > fourth_wave_end):
            covid_growth = covid_growth - ((CURRENT_COVID_DERIVATIVE / growth_derivative_factor) * (1 / monthly_growth_modifier))
        else:
            covid_growth = covid_growth + ((CURRENT_COVID_DERIVATIVE * growth_derivative_factor) * monthly_growth_modifier)
        covid_growth = np.round(covid_growth, 0)
        old_covid_growth = covid_growth
        if old_covid_growth < 0:
            old_covid_growth = 0
            
        covid_growth -= np.round(vaccine_rate * 0.02, 0)
        if covid_growth < 0:
            covid_growth = 0
            
        random_case_fluctuation = np.round(normal_sample(RANDOM_CASE_FLUCTUATION_LOW, RANDOM_CASE_FLUCTUATION_HIGH, 0.8), 0)
        covid = covid + covid_growth + random_case_fluctuation
            
        if really_verbose:
            print('...Total COVID cases now {} (+{} {} -{} *{})'.format(covid,
                                                                        old_covid_growth,
                                                                        "%+d" % random_case_fluctuation,
                                                                        np.round(vaccine_rate * 0.02, 0),
                                                                        monthly_growth_modifier))
                
        if not fourth_wave and np.random.random() <= DAILY_FOURTH_WAVE_CHANCE:
            if really_verbose:
                print('...COVID FOURTH WAVE!!!')
            fourth_wave = True
            fourth_wave_end = day + fourth_wave_days_length
            
            
        # Record
        if date == '2020-12-07':
            dec7_covid = covid
            if verbose:
                print('...COVID by 7 Dec: {}'.format(dec7_covid))
                    
        if date == '2020-12-31':
            dec_2020_vaccinated = total_vaccinated
            q4_2020_covid = covid - Q4_2020_START_COVID
            if verbose:
                print('...Q4 2020 COVID: {}'.format(q4_2020_covid))
            y2020_covid = np.round(covid * missing_infection_rate, 0)
            if verbose:
                print('...Total 2020 COVID: {}'.format(covid))
                print('...Total 2020 COVID after missing transform: {}'.format(y2020_covid))
                
        if date == '2021-01-30':
            jan_covid = covid - q4_2020_covid - Q4_2020_START_COVID
            jan_vaccinated = total_vaccinated
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-02-28':
            feb_covid = covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            feb_vaccinated = total_vaccinated
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-03-31':
            mar_covid = covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            q1_2021_covid = covid - q4_2020_covid - Q4_2020_START_COVID
            mar_vaccinated = total_vaccinated
            if verbose:
                print('...Q1 2021 COVID: {}'.format(q1_2021_covid))
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
            
        if date == '2021-04-30':
            apr_vaccinated = total_vaccinated
            apr_covid = covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-05-31':
            may_vaccinated = total_vaccinated
            may_covid = covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
            
        if date == '2021-06-30':
            jun_vaccinated = total_vaccinated
            jun_covid = covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
            
        if date == '2021-07-31':
            jul_vaccinated = total_vaccinated
            jul_covid = covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-08-31':
            aug_vaccinated = total_vaccinated
            aug_covid = covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
            
        if date == '2021-09-30':
            sep_vaccinated = total_vaccinated
            sep_covid = covid - aug_covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-10-31':
            oct_vaccinated = total_vaccinated
            oct_covid = covid - sep_covid - aug_covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
            
        if date == '2021-11-30':
            nov_vaccinated = total_vaccinated
            nov_covid = covid - oct_covid - sep_covid - aug_covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
            
        if date == '2021-12-31':
            dec_vaccinated = total_vaccinated
            dec_covid = covid - nov_covid - oct_covid - sep_covid - aug_covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
   
    dec_7_2020_covid_collector.append(dec7_covid)
    q4_2020_covid_collector.append(q4_2020_covid)
    y2020_covid_collector.append(y2020_covid)
    q1_2021_covid_collector.append(q1_2021_covid)
    jan_covid_collector.append(jan_covid)
    feb_covid_collector.append(feb_covid)
    mar_covid_collector.append(mar_covid)
    apr_covid_collector.append(apr_covid)
    may_covid_collector.append(may_covid)
    jun_covid_collector.append(jun_covid)
    jul_covid_collector.append(jul_covid)
    aug_covid_collector.append(aug_covid)
    sep_covid_collector.append(sep_covid)
    oct_covid_collector.append(oct_covid)
    nov_covid_collector.append(nov_covid)
    dec_covid_collector.append(dec_covid)
    dec_2020_vaccinated_collector.append(dec_2020_vaccinated)
    jan_vaccinated_collector.append(jan_vaccinated)
    feb_vaccinated_collector.append(feb_vaccinated)
    mar_vaccinated_collector.append(mar_vaccinated)
    apr_vaccinated_collector.append(apr_vaccinated)
    may_vaccinated_collector.append(may_vaccinated)
    jun_vaccinated_collector.append(jun_vaccinated)
    jul_vaccinated_collector.append(jul_vaccinated)
    aug_vaccinated_collector.append(aug_vaccinated)
    sep_vaccinated_collector.append(sep_vaccinated)
    oct_vaccinated_collector.append(oct_vaccinated)
    nov_vaccinated_collector.append(nov_vaccinated)
    dec_vaccinated_collector.append(dec_vaccinated)


-
-
## SCENARIO 1/1000 ##
Growth derivative factor: 3.4
Vaccine modifier: 1.6
Peak growth on: 2021-01-14
Missing infection rate: 16.8
Vaccine delay days: 32.5
News article delay days: 7.0
Day 0: 2020-12-03
...Total vaccinated now 0 (+0)
...Total COVID cases now 62458781.0 (+621748.0 +37033 -0.0 *3.7)
Day 1: 2020-12-04
...Total vaccinated now 0 (+0)
...Total COVID cases now 63194784.0 (+672068.0 +63935 -0.0 *3.7)
Day 2: 2020-12-05
...Total vaccinated now 0 (+0)
...Total COVID cases now 63960006.0 (+722388.0 +42834 -0.0 *3.7)
Day 3: 2020-12-06
...Total vaccinated now 0 (+0)
...Total COVID cases now 64732239.0 (+772708.0 -475 -0.0 *3.7)
Day 4: 2020-12-07
...Total vaccinated now 0 (+0)
...Total COVID cases now 65558299.0 (+823028.0 +3032 -0.0 *3.7)
...COVID by 7 Dec: 65558299.0
Day 5: 2020-12-08
...Total vaccinated now 0 (+0)
...Total COVID cases now 66406041.0 (+873348.0 -25606 -0.0 *3.7)
Day 6: 2020-12-09
...Total vaccinated now 0 (+0)
...Total COVID cases now 67353232.0 (+923668.0 +2352

...Total vaccinated now 1069165392.0 (+10806792.0)
...Total COVID cases now 445552889.0 (+0 +26209 -216136.0 *3.8)
Day 339: 2021-11-07
...Total vaccinated now 1079972184.0 (+10806792.0)
...Total COVID cases now 445589878.0 (+0 +36989 -216136.0 *3.8)
Day 340: 2021-11-08
...Total vaccinated now 1090778976.0 (+10806792.0)
...Total COVID cases now 445597525.0 (+0 +7647 -216136.0 *3.8)
Day 341: 2021-11-09
...Total vaccinated now 1101585768.0 (+10806792.0)
...Total COVID cases now 445606326.0 (+0 +8801 -216136.0 *3.8)
Day 342: 2021-11-10
...Total vaccinated now 1112392560.0 (+10806792.0)
...Total COVID cases now 445625030.0 (+0 +18704 -216136.0 *3.8)
Day 343: 2021-11-11
...Total vaccinated now 1123199352.0 (+10806792.0)
...Total COVID cases now 445678765.0 (+0 +53735 -216136.0 *3.8)
Day 344: 2021-11-12
...Total vaccinated now 1134006144.0 (+10806792.0)
...Total COVID cases now 445697373.0 (+0 +18608 -216136.0 *3.8)
Day 345: 2021-11-13
...Total vaccinated now 1144812936.0 (+10806792.0)
...Tot

Day 249: 2021-08-09
...Total vaccinated now 659594776.0 (+9855188.0)
...Total COVID cases now 99877026.0 (+27720.0 +3231 -197104.0 *3.3)
Day 250: 2021-08-10
...Total vaccinated now 669449964.0 (+9855188.0)
...Total COVID cases now 99892789.0 (+27720.0 +15763 -197104.0 *3.3)
Day 251: 2021-08-11
...Total vaccinated now 679305152.0 (+9855188.0)
...Total COVID cases now 99907624.0 (+27720.0 +14835 -197104.0 *3.3)
Day 252: 2021-08-12
...Total vaccinated now 689160340.0 (+9855188.0)
...Total COVID cases now 99913718.0 (+27720.0 +6094 -197104.0 *3.3)
Day 253: 2021-08-13
...Total vaccinated now 699015528.0 (+9855188.0)
...Total COVID cases now 99952775.0 (+27720.0 +39057 -197104.0 *3.3)
Day 254: 2021-08-14
...Total vaccinated now 708870716.0 (+9855188.0)
...Total COVID cases now 99991690.0 (+27720.0 +38915 -197104.0 *3.3)
Day 255: 2021-08-15
...Total vaccinated now 718725904.0 (+9855188.0)
...Total COVID cases now 99998638.0 (+27720.0 +6948 -197104.0 *3.3)
Day 256: 2021-08-16
...Total vaccinat

...Total vaccinated now 194832009.0 (+2822866.0)
...Total COVID cases now 156847803.0 (+0 +15170 -56457.0 *2.4)
Day 178: 2021-05-30
...Total vaccinated now 197654875.0 (+2822866.0)
...Total COVID cases now 156876651.0 (+0 +28848 -56457.0 *2.4)
Day 179: 2021-05-31
...Total vaccinated now 200477741.0 (+2822866.0)
...Total COVID cases now 156885282.0 (+0 +8631 -56457.0 *2.4)
Day 180: 2021-06-01
...Total vaccinated now 203300607.0 (+2822866.0)
...Total COVID cases now 156896434.0 (+0 +11152 -56457.0 *4.2)
Day 181: 2021-06-02
...Total vaccinated now 206501820.0 (+3201213.0)
...Total COVID cases now 156910844.0 (+0 +14410 -64024.0 *4.2)
Day 182: 2021-06-03
...Total vaccinated now 209703033.0 (+3201213.0)
...Total COVID cases now 156925398.0 (+0 +14554 -64024.0 *4.2)
Day 183: 2021-06-04
...Total vaccinated now 212904246.0 (+3201213.0)
...Total COVID cases now 156946145.0 (+0 +20747 -64024.0 *4.2)
Day 184: 2021-06-05
...Total vaccinated now 216105459.0 (+3201213.0)
...Total COVID cases now 156

Day 106: 2021-03-19
...Total vaccinated now 4009511.0 (+113752.0)
...Total COVID cases now 121406495.0 (+436569.0 +28095 -2275.0 *2.9)
Day 107: 2021-03-20
...Total vaccinated now 4123263.0 (+113752.0)
...Total COVID cases now 121887379.0 (+433763.0 +49396 -2275.0 *2.9)
Day 108: 2021-03-21
...Total vaccinated now 4237015.0 (+113752.0)
...Total COVID cases now 122289497.0 (+430957.0 -26564 -2275.0 *2.9)
Day 109: 2021-03-22
...Total vaccinated now 4350767.0 (+113752.0)
...Total COVID cases now 122718668.0 (+428151.0 +3295 -2275.0 *2.9)
Day 110: 2021-03-23
...Total vaccinated now 4464519.0 (+113752.0)
...Total COVID cases now 123178711.0 (+425345.0 +36973 -2275.0 *2.9)
Day 111: 2021-03-24
...Total vaccinated now 4578271.0 (+113752.0)
...Total COVID cases now 123637731.0 (+422539.0 +38756 -2275.0 *2.9)
Day 112: 2021-03-25
...Total vaccinated now 4692023.0 (+113752.0)
...Total COVID cases now 124060682.0 (+419733.0 +5493 -2275.0 *2.9)
Day 113: 2021-03-26
...Total vaccinated now 4805775.0 (+1

...

...Total COVID cases now 116429346.0 (+0 +34739 -184649.0 *2.2)
Day 367: 2021-12-05
...Total vaccinated now 1092855022.0 (+9232452.0)
...Total COVID cases now 116439104.0 (+0 +9758 -184649.0 *2.2)
Day 368: 2021-12-06
...Total vaccinated now 1102087474.0 (+9232452.0)
...Total COVID cases now 116439578.0 (+0 +474 -184649.0 *2.2)
Day 369: 2021-12-07
...Total vaccinated now 1111319926.0 (+9232452.0)
...Total COVID cases now 116469950.0 (+0 +30372 -184649.0 *2.2)
Day 370: 2021-12-08
...Total vaccinated now 1120552378.0 (+9232452.0)
...Total COVID cases now 116489392.0 (+0 +19442 -184649.0 *2.2)
Day 371: 2021-12-09
...Total vaccinated now 1129784830.0 (+9232452.0)
...Total COVID cases now 116541351.0 (+0 +51959 -184649.0 *2.2)
Day 372: 2021-12-10
...Total vaccinated now 1139017282.0 (+9232452.0)
...Total COVID cases now 116554588.0 (+0 +13237 -184649.0 *2.2)
Day 373: 2021-12-11
...Total vaccinated now 1148249734.0 (+9232452.0)
...Total COVID cases now 116548726.0 (+0 -5862 -184649.0 *2.2)
Da

-
-
## SCENARIO 121/1000 ##
Growth derivative factor: 4.1
Vaccine modifier: 1.1
Peak growth on: 2020-12-03
Missing infection rate: 17.4
Vaccine delay days: 23.2
News article delay days: 2.9
...COVID by 7 Dec: 64817888.0
...Q4 2020 COVID: 53132086.0
...Total 2020 COVID: 79152086.0
...Total 2020 COVID after missing transform: 1377246296.0
...Q1 2021 COVID: 36734395.0
-
-
## SCENARIO 131/1000 ##
Growth derivative factor: 2.4
Vaccine modifier: 13.5
Peak growth on: 2020-12-03
Missing infection rate: 9.9
Vaccine delay days: 25.9
News article delay days: 23.7
...COVID by 7 Dec: 64774151.0
...Q4 2020 COVID: 52936969.0
...Total 2020 COVID: 78956969.0
...Total 2020 COVID after missing transform: 781673993.0
...Q1 2021 COVID: 24636669.0
-
-
## SCENARIO 141/1000 ##
Growth derivative factor: 1.8
Vaccine modifier: 2.3
Peak growth on: 2021-02-23
Missing infection rate: 16.8
Vaccine delay days: 2.7
News article delay days: 12.8
...COVID by 7 Dec: 65025129.0
...Q4 2020 COVID: 60302019.0
...Total 2020 C

-
-
## SCENARIO 351/1000 ##
Growth derivative factor: 3.2
Vaccine modifier: 1.6
Peak growth on: 2020-12-27
Missing infection rate: 49.4
Vaccine delay days: 44.2
News article delay days: 8.2
...COVID by 7 Dec: 64936728.0
...Q4 2020 COVID: 61483668.0
...Total 2020 COVID: 87503668.0
...Total 2020 COVID after missing transform: 4322681199.0
...Q1 2021 COVID: 88599225.0
-
-
## SCENARIO 361/1000 ##
Growth derivative factor: 3.0
Vaccine modifier: 0.8
Peak growth on: 2020-12-03
Missing infection rate: 7.8
Vaccine delay days: 5.6
News article delay days: 8.0
...COVID by 7 Dec: 64923625.0
...Q4 2020 COVID: 53756002.0
...Total 2020 COVID: 79776002.0
...Total 2020 COVID after missing transform: 622252816.0
...Q1 2021 COVID: 34135676.0
-
-
## SCENARIO 371/1000 ##
Growth derivative factor: 2.6
Vaccine modifier: 3.1
Peak growth on: 2021-02-21
Missing infection rate: 6.5
Vaccine delay days: 18.4
News article delay days: 15.8
...COVID by 7 Dec: 65241309.0
...Q4 2020 COVID: 67281094.0
...Total 2020 COVI

-
-
## SCENARIO 581/1000 ##
Growth derivative factor: 2.6
Vaccine modifier: 2.6
Peak growth on: 2020-12-03
Missing infection rate: 13.0
Vaccine delay days: 24.3
News article delay days: 9.0
...COVID by 7 Dec: 64820030.0
...Q4 2020 COVID: 52954625.0
...Total 2020 COVID: 78974625.0
...Total 2020 COVID after missing transform: 1026670125.0
...Q1 2021 COVID: 30169887.0
-
-
## SCENARIO 591/1000 ##
Growth derivative factor: 2.0
Vaccine modifier: 3.3
Peak growth on: 2020-12-03
Missing infection rate: 19.5
Vaccine delay days: 14.6
News article delay days: 8.0
...COVID by 7 Dec: 64910759.0
...Q4 2020 COVID: 53599091.0
...Total 2020 COVID: 79619091.0
...Total 2020 COVID after missing transform: 1552572274.0
...Q1 2021 COVID: 20556395.0
-
-
## SCENARIO 601/1000 ##
Growth derivative factor: 2.4
Vaccine modifier: 2.3
Peak growth on: 2020-12-05
Missing infection rate: 6.7
Vaccine delay days: -2.9
News article delay days: 7.7
...COVID by 7 Dec: 65087023.0
...Q4 2020 COVID: 54225074.0
...Total 2020 CO

-
-
## SCENARIO 811/1000 ##
Growth derivative factor: 3.0
Vaccine modifier: 3.6
Peak growth on: 2021-01-08
Missing infection rate: 6.7
Vaccine delay days: 25.6
News article delay days: 5.9
...COVID by 7 Dec: 65446145.0
...Q4 2020 COVID: 74227876.0
...Total 2020 COVID: 100247876.0
...Total 2020 COVID after missing transform: 671660769.0
...Q1 2021 COVID: 176446871.0
-
-
## SCENARIO 821/1000 ##
Growth derivative factor: 1.8
Vaccine modifier: 1.1
Peak growth on: 2021-01-03
Missing infection rate: 7.5
Vaccine delay days: 24.1
News article delay days: 26.2
...COVID by 7 Dec: 64910364.0
...Q4 2020 COVID: 56471557.0
...Total 2020 COVID: 82491557.0
...Total 2020 COVID after missing transform: 618686678.0
...Q1 2021 COVID: 72898239.0
-
-
## SCENARIO 831/1000 ##
Growth derivative factor: 2.8
Vaccine modifier: 1.3
Peak growth on: 2020-12-21
Missing infection rate: 18.2
Vaccine delay days: 38.1
News article delay days: 7.5
...COVID by 7 Dec: 65206524.0
...Q4 2020 COVID: 66482093.0
...Total 2020 CO

## Total Worldwide COVID by 7 Dec

For calibration

In [4]:
get_metaculus_range(dec_7_2020_covid_collector)

{'lower25': 64900754.5, 'mean': 65040577.0, 'upper75': 65253964.25}

## 2020 Total COVID Cases with Missing Case Estimation

For https://www.metaculus.com/questions/3892/how-many-infections-of-sars-cov-2-novel-coronavirus-will-be-estimated-to-have-occurred-worldwide-before-2021/

In [5]:
get_metaculus_range(y2020_covid_collector, minx=1000000)

{'lower25': 711130124.0,
 'mean': 1086831227.0,
 'upper75': 1743413210.75,
 'minx': '<1.0%'}

## 2020 Q4 Recorded COVID Cases

For https://www.metaculus.com/questions/3767/how-many-new-cases-of-covid-19-in-the-4th-quarter-of-2020/

In [6]:
get_metaculus_range(q4_2020_covid_collector, minx=1000, maxx=1000000000)

{'lower25': 53714134.75,
 'mean': 59426754.0,
 'upper75': 65746996.75,
 'minx': '<1.0%',
 'maxx': '<1.0%'}

## 2021 Q1 Recorded COVID Cases

For https://www.metaculus.com/questions/3768/how-many-new-cases-of-covid-19-in-the-1st-quarter-of-2021/

In [7]:
get_metaculus_range(q1_2021_covid_collector, minx=1000, maxx=1000000000)

{'lower25': 36235908.5,
 'mean': 71158690.5,
 'upper75': 141037392.0,
 'minx': '<1.0%',
 'maxx': '<1.0%'}

## Vaccinations

In [8]:
months = ['Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021', 'Jul 2021',
          'Aug 2021', 'Sep 2021', 'Oct 2021', 'Nov 2021', 'Dec 2021']

collectors = [dec_2020_vaccinated_collector, jan_vaccinated_collector, feb_vaccinated_collector,
              mar_vaccinated_collector, apr_vaccinated_collector, may_vaccinated_collector,
              jun_vaccinated_collector, jul_vaccinated_collector, aug_vaccinated_collector,
              sep_vaccinated_collector, oct_vaccinated_collector, nov_vaccinated_collector,
              dec_vaccinated_collector]

for i, month in enumerate(months):
    rangex = get_metaculus_range(collectors[i])
    print('total vaccinated by EOM {} -- 25% low {}M mean {}M 75% high {}M'.format(month,
                                                                                   np.round(rangex['lower25'] / million, 1),
                                                                                   np.round(rangex['mean'] / million, 1),
                                                                                   np.round(rangex['upper75'] / million, 1)))

total vaccinated by EOM Dec 2020 -- 25% low 0.0M mean 0.0M 75% high 0.3M
total vaccinated by EOM Jan 2021 -- 25% low 2.1M mean 4.4M 75% high 8.8M
total vaccinated by EOM Feb 2021 -- 25% low 6.5M mean 12.7M 75% high 23.2M
total vaccinated by EOM Mar 2021 -- 25% low 20.0M mean 37.7M 75% high 69.5M
total vaccinated by EOM Apr 2021 -- 25% low 45.3M mean 84.5M 75% high 151.8M
total vaccinated by EOM May 2021 -- 25% low 81.3M mean 147.8M 75% high 267.6M
total vaccinated by EOM Jun 2021 -- 25% low 127.7M mean 233.7M 75% high 416.7M
total vaccinated by EOM Jul 2021 -- 25% low 189.0M mean 343.4M 75% high 603.6M
total vaccinated by EOM Aug 2021 -- 25% low 266.2M mean 482.0M 75% high 848.4M
total vaccinated by EOM Sep 2021 -- 25% low 384.8M mean 689.7M 75% high 1223.8M
total vaccinated by EOM Oct 2021 -- 25% low 547.9M mean 972.7M 75% high 1717.9M
total vaccinated by EOM Nov 2021 -- 25% low 732.4M mean 1282.9M 75% high 2279.2M
total vaccinated by EOM Dec 2021 -- 25% low 948.6M mean 1679.1M 75% hi

In [9]:
# https://www.metaculus.com/questions/4066/10m-are-administered-an-efficacious-vaccine/
for i, month in enumerate(months):
    print('10M vaccinated by EOM {}: {}'.format(month, print_pct(np.sum([x > 10 * million for x in collectors[i]]) / len(collectors[i]))))

# https://www.metaculus.com/questions/5727/100m-people-vaccinated-against-covid-19/
print('-')
for i, month in enumerate(months):
    print('100M vaccinated by EOM {}: {}'.format(month, print_pct(np.sum([x > 100 * million for x in collectors[i]]) / len(collectors[i]))))

# https://www.metaculus.com/questions/5767/1bn-people-vaccinated-against-covid-19/
print('-')
for i, month in enumerate(months):
    print('1B vaccinated by EOM {}: {}'.format(month, print_pct(np.sum([x > 1000 * million for x in collectors[i]]) / len(collectors[i]))))

10M vaccinated by EOM Dec 2020: 2.0%
10M vaccinated by EOM Jan 2021: 21.0%
10M vaccinated by EOM Feb 2021: 58.0%
10M vaccinated by EOM Mar 2021: 92.0%
10M vaccinated by EOM Apr 2021: 99.0%
10M vaccinated by EOM May 2021: >99.0%
10M vaccinated by EOM Jun 2021: >99.0%
10M vaccinated by EOM Jul 2021: >99.0%
10M vaccinated by EOM Aug 2021: >99.0%
10M vaccinated by EOM Sep 2021: >99.0%
10M vaccinated by EOM Oct 2021: >99.0%
10M vaccinated by EOM Nov 2021: >99.0%
10M vaccinated by EOM Dec 2021: >99.0%
-
100M vaccinated by EOM Dec 2020: <1.0%
100M vaccinated by EOM Jan 2021: <1.0%
100M vaccinated by EOM Feb 2021: 2.0%
100M vaccinated by EOM Mar 2021: 13.0%
100M vaccinated by EOM Apr 2021: 42.0%
100M vaccinated by EOM May 2021: 66.0%
100M vaccinated by EOM Jun 2021: 84.0%
100M vaccinated by EOM Jul 2021: 92.0%
100M vaccinated by EOM Aug 2021: 98.0%
100M vaccinated by EOM Sep 2021: 99.0%
100M vaccinated by EOM Oct 2021: >99.0%
100M vaccinated by EOM Nov 2021: >99.0%
100M vaccinated by EOM Dec 2

## 2021 Peak Month

For https://www.metaculus.com/questions/5432/month-with-peak-of-covid-19-cases-in-2021/

In [10]:
max_month_collector = []

for i in range(N_SCENARIOS):
    covid_months = [jan_covid_collector[i], feb_covid_collector[i], mar_covid_collector[i], apr_covid_collector[i],
                    may_covid_collector[i], jun_covid_collector[i], jul_covid_collector[i], aug_covid_collector[i],
                    sep_covid_collector[i], oct_covid_collector[i], nov_covid_collector[i], dec_covid_collector[i]]
    max_month = months[covid_months.index(max(covid_months)) + 1]
    max_month_collector.append(max_month)

pd.Series(max_month_collector).value_counts(normalize=True) * 100

Jan 2021    69.7
Mar 2021    12.8
Feb 2021    11.8
May 2021     2.2
Apr 2021     1.6
Jun 2021     0.9
Jul 2021     0.5
Aug 2021     0.3
Sep 2021     0.1
Oct 2021     0.1
dtype: float64